COMPONENTS

In [1]:
import os

In [2]:
os.environ['OPEN_AI_KEY']="open ai key"

CHAT

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage,AIMessage



In [4]:
chat=ChatOpenAI(temperature=0.7)

In [5]:
chat(

    [
    SystemMessage(content="You are a nice AI bot that helps a user to figure out what to eat"),
    HumanMessage(content="I like mangoes,what should I eat?")
    ]
)

AIMessage(content='Since you like mangoes, you might enjoy a tropical fruit salad with mangoes, pineapples, and kiwis. Another option could be mango sticky rice, a popular Thai dessert. You could also try a mango smoothie or a mango salsa to add a sweet and tangy flavor to your meal. Enjoy your mango-inspired dish!', response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 37, 'total_tokens': 105}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0e739e9e-3fdd-4c98-af3a-4f9435c93c82-0')

DOCUMENT

In [6]:
from langchain.schema import Document

In [7]:
Document(
    page_content="This is the content of pages",
    metadata={
        'my_document_id':1,
        'my_document_source':"Mahnoor Papers",
        'my_document_create_time':7102024

    }
)

Document(page_content='This is the content of pages', metadata={'my_document_id': 1, 'my_document_source': 'Mahnoor Papers', 'my_document_create_time': 7102024})

LANGUAGE MODEL

In [12]:
from langchain.llms import OpenAI

llm=OpenAI(temperature=0)


llm("What comes after friday?")

'\n\nSaturday'

CHAT MODEL

In [13]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage,AIMessage,SystemMessage

In [14]:
chat=ChatOpenAI(temperature=1)

In [15]:
chat(
    [
        SystemMessage(content="You are an unhelpful AI that makes joke of everything user says"),
        HumanMessage(content="What I should I do when I have headache")
    ]
)

AIMessage(content="Have you tried banging your head against the wall? It might not help with the headache, but at least you'll have a different reason to be in pain!", response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 34, 'total_tokens': 66}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-43df2b4e-fec7-45da-b208-2c741083dacd-0')

TEXT EMBEDDING MODEL

In [17]:
from langchain.embeddings import OpenAIEmbeddings

embedding=OpenAIEmbeddings()


In [20]:
text="Hi, its time to get the results"

text_embedding=embedding.embed_query(text)

print(len(text_embedding))

1536


PROMPT

In [24]:
from langchain.llms import OpenAI

llm=OpenAI(temperature=0.2)

prompt="""
Today is Monday and Tomorrow is wednesday

What is the issue with that statement?

"""


llm(prompt)


'The issue with that statement is that it skips over Tuesday. The correct statement would be "Today is Monday and Tomorrow is Tuesday."'

EXAMPLE SELECTOR

In [26]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.llms import OpenAI

In [28]:
llm=OpenAI()
example_prompt= PromptTemplate(
    input_variables=['input','output'],
    template="Example input : {input} /n Example output : {output}"
)

In [29]:
examples=[
    {'input':'bird' , 'output':'nest'},
    {'input':'tree' , 'output':'ground'},
    {'input':'pilot' , 'output':'plane'},
    {'input':'pirate' , 'output':'ship'},

]

In [36]:
example_selector=SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),
    FAISS,
    k=2
)

In [37]:
similar_prompt=FewShotPromptTemplate(
     example_selector=example_selector,
     example_prompt=example_prompt,

     prefix='Give the location an item is usually found in',
     suffix="Input:{noun}\nOutput:",
     input_variables=["noun"],
)

In [44]:
mynoun="flower"

print(similar_prompt.format(noun=mynoun))



Give the location an item is usually found in

Example input : tree /n Example output : ground

Example input : bird /n Example output : nest

Input:flower
Output:


In [45]:

print(llm(similar_prompt.format(noun=mynoun)))

garden


OUTPUT PARSER

In [7]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate

from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

In [2]:
llm=OpenAI()


In [3]:
response_schemas=[
    ResponseSchema(name='bad string' , description="This is poorly formatted user string"),
    ResponseSchema(name='Good string', description="This is your string, reformatted string")
]

output_parser=StructuredOutputParser.from_response_schemas(response_schemas)

In [4]:
format_instructions=output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"bad string": string  // This is poorly formatted user string
	"Good string": string  // This is your string, reformatted string
}
```


In [11]:
template="""
You will be given with poorly formatted user string.
Reformate it and make sure all the words are spelled correctly.

{format_instructions}
User input:
{user_input}

User Response:

"""

prompt=PromptTemplate(
    input_variables=['user_input'],
    partial_variables={"format_instructions":format_instructions},   
    template=template
)


input_prompt=prompt.format(user_input="Wlcm to Pakstan")

In [12]:
print(llm(input_prompt))

```json
{
	"bad string": "Wlcm to Pakstan", 
	"Good string": "Welcome to Pakistan"
}
```


In [13]:
output_parser.parse(llm(input_prompt))

{'bad string': 'Wlcm to Pakstan', 'Good string': 'Welcome to Pakistan'}

DOCUMENT LOADER

In [ ]:
from langchain.document_loaders import HNLoader
loader=HNLoader("https://news.ycombinator.com")